# Imports and types

In [13]:
import random
import time
import numpy as np
import pandas as pd

In [14]:
def calcular_aptidao(individuo):
    return sum(individuo)

In [15]:
def mutacao(solucao):
    nova = solucao[:]
    a, b = random.sample(range(len(nova)), 2)
    nova[a], nova[b] = nova[b], nova[a]
    return nova

In [16]:
def gerar_vizinho(solucao):
    return mutacao(solucao)

In [17]:
def gerar_array_replicavel(seed: int, tamanho: int) -> list[int]:
    random.seed(seed)
    vetor = list(range(tamanho))  
    random.shuffle(vetor)
    return vetor

In [18]:
def calcular_aptidao_qap(solucao, matriz_fluxo, matriz_distancia):
    n = len(solucao)
    custo = 0
    for i in range(n):
        for j in range(n):
            custo += matriz_fluxo[i][j] * matriz_distancia[solucao[i]][solucao[j]]
    return custo

In [19]:
def VND(solucao, matriz_fluxo, matriz_distancia, kmax=5):
    melhor = solucao[:]
    k = 1
    while k <= kmax:
        vizinho = gerar_vizinho(melhor)
        if calcular_aptidao_qap(vizinho, matriz_fluxo, matriz_distancia) < calcular_aptidao_qap(melhor, matriz_fluxo, matriz_distancia):
            melhor = vizinho
            k = 1
        else:
            k += 1
    return melhor

In [20]:
def ES_VND(mu, lambd, tempo_max, taxa_mutacao, taxa_busca_local, iter_sem_melhora_max,
           solucao_inicial, matriz_fluxo, matriz_distancia):

    P = [mutacao(solucao_inicial) for _ in range(lambd)]
    melhor = None
    melhor_aptidao = float('inf')
    sem_melhora = 0

    inicio = time.time()

    while (time.time() - inicio) < tempo_max and sem_melhora < iter_sem_melhora_max:
        aptidoes = [calcular_aptidao_qap(ind, matriz_fluxo, matriz_distancia) for ind in P]

        for ind, apt in zip(P, aptidoes):
            if apt < melhor_aptidao:
                melhor_aptidao = apt
                melhor = ind[:]
                sem_melhora = 0
        sem_melhora += 1

        melhores_indices = sorted(range(len(P)), key=lambda i: aptidoes[i])[:mu]
        Q = [P[i][:] for i in melhores_indices]

        nova_geracao = Q[:]
        for q in Q:
            for _ in range(lambd // mu):
                individuo = q[:]
                if random.random() < taxa_mutacao:
                    individuo = mutacao(individuo)
                if random.random() < taxa_busca_local:
                    individuo = VND(individuo, matriz_fluxo, matriz_distancia)
                nova_geracao.append(individuo)

        P = nova_geracao

    return melhor, melhor_aptidao


In [21]:
def ler_qap_com_n(caminho: str):
    with open(caminho, "r") as f:
        dados = list(map(int, f.read().split()))
        
    n = dados[0]
    valores = dados[1:]  

    total_esperado = 2 * n * n
    if len(valores) != total_esperado:
        raise ValueError(f"Esperado {total_esperado} valores, mas encontrado {len(valores)}.")

    flow_flat = valores[:n * n]
    dist_flat = valores[n * n:]

    flow_df = pd.DataFrame([flow_flat[i * n:(i + 1) * n] for i in range(n)])
    dist_df = pd.DataFrame([dist_flat[i * n:(i + 1) * n] for i in range(n)])

    return n, flow_df, dist_df

In [22]:
if __name__ == "__main__":
    n, flow_df, dist_df = ler_qap_com_n("Chr15b.txt")
    flow = flow_df.values.tolist()
    dist = dist_df.values.tolist()
    matriz_fluxo = np.array(flow)
    matriz_distancia = np.array(dist)
    solucao_inicial = gerar_array_replicavel(seed=42, tamanho=n)

    parametros = {
        "mu": random.randint(5, 10),
        "lambd": random.randint(30, 100),
        "tempo_max": random.randint(3, 10) * 60,
        "taxa_mutacao": random.uniform(0.4, 0.7),
        "taxa_busca_local": random.uniform(0.4, 0.7),
        "iter_sem_melhora_max": random.randint(5, 10),
        "solucao_inicial": solucao_inicial,
        "matriz_fluxo": matriz_fluxo,
        "matriz_distancia": matriz_distancia
    }

    melhor_solucao, melhor_valor = ES_VND(**parametros)
    print("Melhor solução encontrada:", melhor_solucao)
    print("Custo:", melhor_valor)
    print("Parâmetros usados:", parametros)

Melhor solução encontrada: [8, 13, 2, 12, 3, 7, 1, 9, 5, 11, 4, 14, 10, 0, 6]
Custo: 11598
Parâmetros usados: {'mu': 5, 'lambd': 33, 'tempo_max': 240, 'taxa_mutacao': 0.465591392441081, 'taxa_busca_local': 0.5516065864310087, 'iter_sem_melhora_max': 5, 'solucao_inicial': [8, 13, 7, 6, 14, 12, 5, 2, 9, 3, 4, 11, 0, 1, 10], 'matriz_fluxo': array([[ 0, 12, 86, 68,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [12,  0,  0,  0, 69, 34, 35,  0,  0,  0,  0,  0,  0,  0,  0],
       [86,  0,  0,  0,  0,  0,  0, 19,  0,  0,  0,  0,  0,  0,  0],
       [68,  0,  0,  0,  0,  0,  0,  0,  2, 21, 88,  0,  0,  0,  0],
       [ 0, 69,  0,  0,  0,  0,  0,  0,  0,  0,  0, 79,  3, 40,  0],
       [ 0, 34,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 35,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 77],
       [ 0,  0, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 21,  0,  0,  0,  0,